In [1]:
import numpy as np
from qiskit import *

In [2]:
def initialize_circuit():
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    return qc

In [3]:
def zz_measurement(qc):
    """Measure 'qubit' in the ZZ-basis, and store the result in 'cbit'"""
    qc.measure([0,1],[0,1])
    return qc

def zv_measurement(qc):
    """Measure 'qubit' in the ZV-basis, and store the result in 'cbit'"""
    qc.ry(np.pi/4,1)
    qc.measure([0,1],[0,1])
    return qc

def zw_measurement(qc):
    """Measure 'qubit' in the ZW-basis, and store the result in 'cbit'"""
    qc.ry(-np.pi/4,1)
    qc.measure([0,1],[0,1])
    return qc

def xv_measurement(qc):
    """Measure 'qubit' in the XV-basis, and store the result in 'cbit'"""
    qc.h(0)
    qc.ry(+np.pi/4,1)
    qc.measure([0,1],[0,1])
    return qc

def xw_measurement(qc):
    """Measure 'qubit' in the XW-basis, and store the result in 'cbit'"""
    qc.h(0)
    qc.ry(-np.pi/4,1)
    qc.measure([0,1],[0,1])
    return qc


In [4]:
shots = 65536 # number of samples used for statistics

def average(qc, measurement='ZZ'):

    
    if measurement == 'ZW':
        circuit = zw_measurement(qc)
    elif measurement == 'ZV':
        circuit = zv_measurement(qc)
    elif measurement == 'XW':
        circuit = xw_measurement(qc)
    elif measurement == 'XV':
        circuit = xv_measurement(qc)
    else:
        circuit = zz_measurement(qc)
    
    # run the circuit with the selected measurement and 
    # get the number of samples that output each bit value
    job = execute(circuit, Aer.get_backend('qasm_simulator'), shots=shots)
    result = job.result()
    counts = result.get_counts()

    # calculate the probabilities for each bit value
    probs = {}
    for output in ['00', '01', '10', '11']:
        if output in counts:
            probs[output] = counts[output]/shots
        else:
            probs[output] = 0
            
    # calculate average value
    avg = probs['00'] + probs['11'] - probs['10'] - probs['01']
    
    return avg

In [5]:
def S(qc):
    tmp = [QuantumCircuit.copy(qc) for i in range(4)]
    S = average(tmp[0], measurement='ZV') + average(tmp[1], measurement='ZW') + average(tmp[2], measurement='XW') - average(tmp[3], measurement='XV')

    return S

In [6]:
S(initialize_circuit())

2.829315185546875